# Fiesole Control

This project aims to demonstrate the feasibility of Vertical-Federated-Learning CNN architectures.


In [ ]:
%pip install seaborn
%pip install torch
%pip install torchvision

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
trainset = torchvision.datasets.MNIST(
    root="./control", train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(
    root="./control", train=False, download=True, transform=transform
)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [4]:
BATCH_SIZE = 32
MNIST_CLASSES = 10

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc_relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(-1, 64 * 7 * 7)

        x = self.fc1(x)
        x = self.fc_relu1(x)
        x = self.fc2(x)
        return x

In [6]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        # Forward + Backward + Optimize
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training finished!")

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(
        "Test Accuracy of the model on the 10000 test images: {} %".format(
            100 * correct / total
        )
    )

Epoch [1/10], Step [100/938], Loss: 0.2759
Epoch [1/10], Step [200/938], Loss: 0.0639
Epoch [1/10], Step [300/938], Loss: 0.0768
Epoch [1/10], Step [400/938], Loss: 0.3931
Epoch [1/10], Step [500/938], Loss: 0.0443
Epoch [1/10], Step [600/938], Loss: 0.0500
Epoch [1/10], Step [700/938], Loss: 0.0388
Epoch [1/10], Step [800/938], Loss: 0.0114
Epoch [1/10], Step [900/938], Loss: 0.0049
Epoch [1/10], Loss: 0.0288
Epoch [2/10], Step [100/938], Loss: 0.0439
Epoch [2/10], Step [200/938], Loss: 0.0219
Epoch [2/10], Step [300/938], Loss: 0.0851
Epoch [2/10], Step [400/938], Loss: 0.0136
Epoch [2/10], Step [500/938], Loss: 0.0790
Epoch [2/10], Step [600/938], Loss: 0.0072
Epoch [2/10], Step [700/938], Loss: 0.0193
Epoch [2/10], Step [800/938], Loss: 0.0174
Epoch [2/10], Step [900/938], Loss: 0.0135
Epoch [2/10], Loss: 0.0531
Epoch [3/10], Step [100/938], Loss: 0.0400
Epoch [3/10], Step [200/938], Loss: 0.0100
Epoch [3/10], Step [300/938], Loss: 0.0110
Epoch [3/10], Step [400/938], Loss: 0.0296


In [8]:
torch.save(model.state_dict(), "control.ckpt")